In [1]:
import pandas as pd
import numpy as np
from sklearn.externals import joblib
import time
from IPython.display import Image

from sklearn import tree
from dtreeviz.trees import *
import seaborn as sns

pd.set_option('display.max_columns', None)

%pylab inline

Populating the interactive namespace from numpy and matplotlib


/home/user/venvs/MARLENA_env/lib/python3.6/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))


In [2]:
def highlight_max(data, color='yellow'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = 'background-color: {}'.format(color)
    
    if data.ndim == 1:  # Series from .apply(axis=0) or axis=1
        is_max = data == data.max()
        return [attr if v else '' for v in is_max]
    else:  # from .apply(axis=None)
        is_max = data == data.max().max()
        return pd.DataFrame(np.where(is_max, attr, ''),
                            index=data.index, columns=data.columns)

<p>
  <font size="6">$$b{:}\mathcal{X}^{(m)} {\rightarrow} \mathcal{Y}^{(l)}$$</font> 
</p>


# MARLENA: explaining multi-label black-box decisions

### Multi-label classification

Multi-label tasks are quite common, everytime an instance might be associated with more than one not mutually exclusive labels we have a multi-label problem. 


<center><img style="float: ;" src="./data/gitignoreDATA/presentazione_lipari_MARLENA/carlino_multilabel.png"></center>

<center><font size="6">[Dog, Wig, Guitar]</font></center>



Most of the high-performance algorithms are black-boxes. How can we explain a multi-label black box decision using **MARLENA**?

Let's walk through an example.

### Dataset: *NYS OMH Patient Characteristics Survey (PCS) 2015*
It contains demographic, clinical, social, and insurance characteristics for each client served by the NYS public mental health system during the week of October 19, 2015.

<center><img style="width: 60%; height: 50%" src="./data/gitignoreDATA/presentazione_lipari_MARLENA/Data.png"></center>

https://www.kaggle.com/new-york-state/nys-patient-characteristics-survey-pcs-2015

In [3]:
#load the dataset:
df = pd.read_csv('./data/mental_illness.csv',index_col=0)
print(df.shape)
df.head()

(105098, 156)


,Age Group_CHILD,Sex_MALE,Hispanic Ethnicity,Veteran Status,Mental Illness,Intellectual Disability,Autism Spectrum,Other Developmental Disability,Alcohol Related Disorder,Drug Substance Disorder,Mobility Impairment Disorder,Hearing Visual Impairment,Hyperlipidemia,High Blood Pressure,Diabetes,Obesity,Heart Attack,Stroke,Other Cardiac,Pulmonary Asthma,Alzheimer or Dementia,Kidney Disease,Liver Disease,Endocrine Condition,Neurological Condition,Traumatic Brain Injury,Joint Disease,Cancer,Other Chronic Med Condition,No Chronic Med Condition,Smokes,Received Smoking Medication,Received Smoking Counseling,Serious Mental Illness,SSI Cash Assistance,SSDI Cash Assistance,Veterans Disability Benefits,Veterans Cash Assistance,Public Assistance Cash Program,Other Cash Benefits,Medicaid and Medicare Insurance,No Insurance,Medicaid Insurance,Medicare Insurance,Private Insurance,Child Health Plus Insurance,Other Insurance,Criminal Justice Status,Program Category_EMERGENCY,Program Category_INPATIENT,Program Category_OUTPATIENT,Program Category_RESIDENTIAL,Program Category_SUPPORT,Region Served_CENTRAL NY REGION,Region Served_HUDSON RIVER REGION,Region Served_LONG ISLAND REGION,Region Served_NEW YORK CITY REGION,Region Served_WESTERN REGION,Transgender_CLIENT DIDN'T ANSWER,"Transgender_NO, NOT TRANSGENDER","Transgender_YES, TRANSGENDER",Sexual Orientation_BISEXUAL,Sexual Orientation_CLIENT DID NOT ANSWER,Sexual Orientation_LESBIAN OR GAY,Sexual Orientation_OTHER,Sexual Orientation_STRAIGHT OR HETEROSEXUAL,Race_BLACK ONLY,Race_MULTI-RACIAL,Race_OTHER,Race_UNKNOWN RACE,Race_WHITE ONLY,Living Situation_INSTITUTIONAL SETTING,Living Situation_OTHER LIVING SITUATION,Living Situation_PRIVATE RESIDENCE,Household Composition_COHABITATES WITH OTHERS,Household Composition_LIVES ALONE,Household Composition_NOT APPLICABLE,Preferred Language_AFRO-ASIATIC,Preferred Language_ALL OTHER LANGUAGES,Preferred Language_ASIAN AND PACIFIC ISLAND,Preferred Language_ENGLISH,Preferred Language_INDO-EUROPEAN,Preferred Language_SPANISH,Employment Status_EMPLOYED,Employment Status_NON-PAID/VOLUNTEER,Employment Status_NOT IN LABOR FORCE:UNEMPLOYED AND NOT LOOKING FOR WORK,"Employment Status_UNEMPLOYED, LOOKING FOR WORK",Employment Status_UNKNOWN EMPLOYMENT STATUS,Number Of Hours Worked Each Week_01-14 HOURS,Number Of Hours Worked Each Week_15-34 HOURS,Number Of Hours Worked Each Week_35 HOURS OR MORE,Number Of Hours Worked Each Week_NOT APPLICABLE,Number Of Hours Worked Each Week_UNKNOWN EMPLOYMENT HOURS,Education Status_COLLEGE OR GRADUATE DEGREE,Education Status_MIDDLE SCHOOL TO HIGH SCHOOL,Education Status_NO FORMAL EDUCATION,Education Status_OTHER,Education Status_PRE-K TO FIFTH GRADE,Education Status_SOME COLLEGE,Special Education Services_NO,Special Education Services_NOT APPLICABLE,Special Education Services_YES,Medicaid Managed Insurance_NO,Medicaid Managed Insurance_NOT APPLICABLE,Medicaid Managed Insurance_YES,ZIP3=100,ZIP3=101,ZIP3=103,ZIP3=104,ZIP3=105,ZIP3=106,ZIP3=107,ZIP3=108,ZIP3=109,ZIP3=110,ZIP3=111,ZIP3=112,ZIP3=113,ZIP3=114,ZIP3=115,ZIP3=116,ZIP3=117,ZIP3=118,ZIP3=119,ZIP3=120,ZIP3=121,ZIP3=122,ZIP3=123,ZIP3=124,ZIP3=125,ZIP3=126,ZIP3=127,ZIP3=128,ZIP3=129,ZIP3=130,ZIP3=131,ZIP3=132,ZIP3=133,ZIP3=134,ZIP3=135,ZIP3=136,ZIP3=137,ZIP3=138,ZIP3=139,ZIP3=140,ZIP3=141,ZIP3=142,ZIP3=143,ZIP3=144,ZIP3=145,ZIP3=146,ZIP3=147,ZIP3=148,ZIP3=149,ZIP3=888,ZIP3=999
0,0,1,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [4]:
#set the labels
labels = ['Mental Illness',
 'Serious Mental Illness',
 'Other Developmental Disability',
 'Alcohol Related Disorder',
 'Drug Substance Disorder']

df.groupby(labels).size().reset_index().sort_values(by=0,ascending=False).reset_index(drop=True).rename(columns={0:'n'})

,Mental Illness,Serious Mental Illness,Other Developmental Disability,Alcohol Related Disorder,Drug Substance Disorder,n
0,1,1,0,0,0,67248
1,1,0,0,0,0,12889
2,1,1,0,0,1,6837
3,1,1,0,1,1,5756
4,1,1,0,1,0,3116
5,0,0,0,0,0,2574
6,1,1,1,0,0,2247
7,0,1,0,0,0,1674
8,1,0,0,0,1,702
9,1,0,0,1,0,402


### Test some black boxes:

Split in training and test set

In [5]:
X = df.drop(labels,1)
y = df[labels]

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [6]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import dummy

names = ["MLPClassifier",\
         "RandomForestClassifier",\
         "OneVsRestClassifier_SVM",\
         "OneVsRestClassifier_Adaboost",\
         "Dummy Stratified"]

classifiers = [MLPClassifier(),\
              RandomForestClassifier(),\
              OneVsRestClassifier(SVC(gamma='auto')),\
              OneVsRestClassifier(AdaBoostClassifier(n_estimators=1000)),\
              dummy.DummyClassifier(strategy='stratified')]

In [7]:
import warnings
warnings.simplefilter('ignore')

from sklearn.metrics import accuracy_score, \
hamming_loss, \
recall_score, \
precision_score, \
f1_score, \
fbeta_score, \
coverage_error

tabella_metriche_df = pd.DataFrame(columns=['accuracy score',\
                                            '1-hamming_loss',\
                                            'precision score sample',\
                                            'recall score sample',\
                                            'F1-score sample',\
                                            'precision score macro',\
                                            'recall score macro',\
                                            'F1-score macro',\
                                            'precision score micro',\
                                            'recall score micro',\
                                            'F1-score micro'
                                           ], index=names)


for name, clf in zip(names, classifiers):
    print(name)
    
    time_start = time.time()
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test) 
    
    deltaT = time.time() - time_start
    print(f'seconds elapsed: {deltaT}')
    
    joblib.dump(clf, './black_boxes/%s_mental.pkl' % name) 
    y_true = y_test
    
    tabella_metriche_df.loc[name] = pd.Series({'accuracy score':round(accuracy_score(y_true, y_pred),3),\
                                                      '1-hamming_loss':round(1-hamming_loss(y_true, y_pred),3),\
                                                      'precision score sample':round(precision_score(y_true, y_pred,average='samples'),3),\
                                                      'recall score sample':round(recall_score(y_true, y_pred,average='samples'),3),\
                                                      'F1-score sample':round(f1_score(y_true, y_pred,average='samples'),3),
                                                      'precision score macro':round(precision_score(y_true, y_pred,average='macro'),3),
                                                      'recall score macro':round(recall_score(y_true, y_pred,average='macro'),3),
                                                      'F1-score macro':round(f1_score(y_true, y_pred,average='macro'),3),
                                                      'precision score micro':round(precision_score(y_true, y_pred,average='micro'),3),
                                                      'recall score micro':round(recall_score(y_true, y_pred,average='micro'),3),
                                                      'F1-score micro':round(f1_score(y_true, y_pred,average='micro'),3)})
    

MLPClassifier
seconds elapsed: 191.4045534133911
RandomForestClassifier
seconds elapsed: 2.7412028312683105
OneVsRestClassifier_SVM
seconds elapsed: 11648.33759021759
OneVsRestClassifier_Adaboost
seconds elapsed: 326.55746722221375
Dummy Stratified
seconds elapsed: 0.01080012321472168


In [ ]:
tabella_metriche_df.style.apply(highlight_max)

<center><img style="float: ;" src="./data/gitignoreDATA/presentazione_lipari_MARLENA/metriche.png"></center>

So now we have the black box

In [9]:
bb = joblib.load('./black_boxes/OneVsRestClassifier_Adaboost_mental.pkl')

And now... **MARLENA**

https://github.com/CeciPani/MARLENA

<center><img style="width: 60%; height: 60%" src="./data/gitignoreDATA/presentazione_lipari_MARLENA/2_outcome_explanation.png"></center>

1. It generates a synthetic neighborhood around the instance to be explained using a strategy suitable for multi-label decisions.
2. It learns a decision tree (DT) on such neighborhood.
3. It derives from the DT a decision rule that explains the black box decision.

### How MARLENA builds the synthetic neighborhood

Suppose we want to explain this instance

<center><img style="width: 60%; height: 60%" src="./data/gitignoreDATA/presentazione_lipari_MARLENA/dataset_sample.png"></center>

### MARLENA starts from a the feature distribution of the closest neighbors of the instance

<center><img style="width: 60%; height: 60%" src="./data/gitignoreDATA/presentazione_lipari_MARLENA/mixed_real_neigh.png"></center>

### Selecting the real neighbors: feature space

A fraction $\alpha$ of these neighbors are the closest in the feature space according to a distance function $d_f(x,\hat{x})$
<center><img style="width: 60%; height: 60%" src="./data/gitignoreDATA/presentazione_lipari_MARLENA/mixed_features_real_neigh.png"></center> 

### Selecting the real neighbors: label space

A fraction $1-\alpha$ of these neighbors are the closest in the **label** space according to a distance function $d_l(b(x),b(\hat{x}))$

<center><img style="width: 60%; height: 60%" src="./data/gitignoreDATA/presentazione_lipari_MARLENA/mixed_labels_real_neigh.png"></center>

### MARLENA then create the synthetic neighbors perturbing the real ones.

From the distribution of the features of these real neighbors MARLENA creates the synthetic ones. Then the black box is used to label them to generate a new **ground truth** for the interpretable classifier (DT).

<center><img style="width: 70%; height: 70%" src="./data/gitignoreDATA/presentazione_lipari_MARLENA/labeling.png"></center>

In [10]:
from marlena import marlena

Istantiate the MARLENA object

In [11]:
m1 = marlena.MARLENA(neigh_type='mixed')

In [12]:
#select one instance whose multi-label decision we want to explain:
instance_to_be_explained = df.drop(labels,1).loc[16]

# black box decision:
print(f'black box prediction: {bb.predict(np.array(instance_to_be_explained.values).reshape(1, -1))}')

#remember that:
labels

black box prediction: [[1 1 0 0 1]]


['Mental Illness',
 'Serious Mental Illness',
 'Other Developmental Disability',
 'Alcohol Related Disorder',
 'Drug Substance Disorder']

In [13]:
#this patient features
instance_to_be_explained[instance_to_be_explained!=0]

Sex_MALE                                                                    1
Hyperlipidemia                                                              1
Obesity                                                                     1
Endocrine Condition                                                         1
Traumatic Brain Injury                                                      1
Smokes                                                                      1
Received Smoking Medication                                                 1
Received Smoking Counseling                                                 1
Medicaid Insurance                                                          1
Program Category_INPATIENT                                                  1
Region Served_NEW YORK CITY REGION                                          1
Transgender_NO, NOT TRANSGENDER                                             1
Sexual Orientation_STRAIGHT OR HETEROSEXUAL                     

To extract the explanation, MARLENA needs:

In [14]:
# the black box:
bb = joblib.load('./black_boxes/OneVsRestClassifier_Adaboost_mental.pkl')

In [15]:
# the instance to be explained:
i2e = instance_to_be_explained

In [16]:
#the name of the classes:
labels_name = labels
#which variables are categorical and which are numerical:
numerical_vars = []
categorical_vars = df.drop(labels,1).columns.values

In [17]:
# a set of instances representative of the dataset that was used to train the black box 
X2E = df.drop(labels_name,1)

#### MARLENA hyperparameters:
* **k**: number of real neighbors to use to generate the synthetic ones 
* **alpha**: fraction of neighbors to be sampled from the feature space
* **k_synth**: number of synthetic neighbors

In [18]:
k = 50
k_synt = 1000
alpha = 0.7

In [19]:
labels_name = ['Mental Illness',
 'Serious Mental Illness',
 'Other Developmental Disability',
 'Alcohol Related Disorder',
 'Drug Substance Disorder']

### MARLENA "extract_explanation" method:

In [20]:
rule, instance_imporant_feat, fidelity, hit, DT = m1.extract_explanation(i2e,\
                                                                         X2E,\
                                                                         bb,\
                                                                         numerical_vars,\
                                                                         categorical_vars,\
                                                                         labels_name,\
                                                                         k=k,\
                                                                         size=k_synt,\
                                                                         alpha=alpha)

MARLENA-mixed
decision rule: {Smokes = True,
 Living Situation_INSTITUTIONAL SETTING = False,
 SSI Cash Assistance = False,
 Region Served_LONG ISLAND REGION = False,
 Household Composition_COHABITATES WITH OTHERS = False,
 Traumatic Brain Injury = True,
 ZIP3=112 = False} -> ['Mental Illness' 'Serious Mental Illness' 'Drug Substance Disorder']
rule length: 7
black-box decision: [[1 1 0 0 1]]
fidelity of DT: 0.908125179443009
hit: 1.0


### Marlena output:

In [21]:
#decision rule
print(rule)

{Smokes = True,
 Living Situation_INSTITUTIONAL SETTING = False,
 SSI Cash Assistance = False,
 Region Served_LONG ISLAND REGION = False,
 Household Composition_COHABITATES WITH OTHERS = False,
 Traumatic Brain Injury = True,
 ZIP3=112 = False} -> ['Mental Illness' 'Serious Mental Illness' 'Drug Substance Disorder']


* INSTITUTIONAL SETTING: Residential Services are provided to maximize access to housing opportunities, particularly for persons with histories of repeated psychiatric hospitalizations, homelessness, involvement with the criminal justice system
* Supplemental Security Income (SSI) is a Federal income supplement program funded by general tax revenues

In [22]:
#fidelity
print(fidelity)

0.908125179443009


$$fidelity(Y,\hat{Y})\in\left[0,1\right]$$

The **fidelity** compares the decisions of the DT to those of the black-box on the synthetic neighborhood. It is measured using the *F1-score*.

In [23]:
#hit
print(hit)

1.0


$$hit(y,\hat{y})) = 1 − hamming(y, ŷ) \in \left[0,1\right]$$

The **hit** compares the predictions of the DT and the black-box on the instance *x* under analysis. It is mesured usinf the *simple match similarity* 

# Thank you